# Environment

## Install Dependencies, store credentials and prep environment


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

## install poetry dependency management

Running poetry commands without the --no-ansi option fails and does not install the dependencies. So `!poetry --no-ansi install` and `!poetry --no-ansi add package` should be used instead which also correctly updates the pyproject.toml file.

In [ ]:
%cd /content/gdrive/MyDrive/
!rm -rf test-poetry
!mkdir test-poetry
%cd test-poetry

## install libraries

In [ ]:
!pip install deepgram-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
!apt install python3-pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 python3-pyaudio
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 91.2 kB of archives.
After this operation, 340 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 91.2 kB in 0s (239 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package python3-pyaudio.
Prepa

In [ ]:
!pip install ibm-watson-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.6 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.2-py3-none-any.whl size=73905 sha256=7065b76efc53fe69fc08b8e0d2fdd32029eb06c01516b3c0b7965c233910dc28
  Stored in directory: /root/.cache/pip/wheels/02/74/84/b23401241f593c79d38bb1e1b0f788e7744aa07338e6382421
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.13.2-py3-none-any.whl size=634575 sha256=734e7aae690ce18bf9576a469fb2bb102ec02ea43da1822c83f735d581169166
  Stored in directory: /root/.cac

In [ ]:
!pip install black

In [ ]:
!pip install "black[jupyter]"

In [ ]:
jupyter nbextension enable black/extension

## import dependencies and set global variables

In [ ]:
import deepgram
import pyaudio
import ibm_watson_machine_learning.foundation_models as foundation_models
import doctest

WATSON_REQUEST_OBJECT = {}
WATSON_API_KEY = "" # @TODO store this in .env and import from local Google drive, DON'T FORGET!


# Phase 1 Implementation:

In [2]:
#@title User Configuration

user_name = 'Anthem' #@param {type:"string"}
expertise_level =  'beginner' #@param ["beginner", "novice", "expert"] {allow-input: true}
learning_depth = 'deep dive' #@param ["overview", "detailed", "deep dive"] {allow-input: true}
time_in_minutes = '30' #@param [5, 10, 15, 30, 45, 60] {allow-input: true}

iq_config = {
    "config": {
        "version": "1.0",
        "purpose": "Human-AI Collaboration for Skill Transfer and Skill Adoption",
        "description": "This configuration is designed to optimize the collaboration between a user and an AI agent for the purpose of learning and mastery of specific subject matter."
    },
    "user": {
        "name": "{user_name}",
        "expertise_level": "{expertise_level}",
        "learning_depth": "{learning_depth}",
        "time_available": "{time_in_minutes}"
    },
    "collaboration": {
        "id": "{auto-incremented based on sessions}",
        "title": "{determined by determineTitle function}",
        "learning_outcome": "{formulated by formulateLearningOutcome function}",
        "tasks": "{devised by deviseTasks function}",
        "feedback": {
            "session_id": "{session_id}",
            "rating": "{user_rating}",
            "comments": "{user_comments}",
            "suggestions": "{user_suggestions}"
        },
        "functions": {
            "incrementSessionId": "Calculate the number of sessions already present and increment by one for the new session.",
            "determineTitle": "Based on your instructional needs, interest, and expertise level, I suggest the title: '{sessionTitle}'. Does that sound appropriate?",
            "formulateLearningOutcome": "Considering your inputs, the learning outcome for this session will be: '{learningOutcome}'",
            "deviseTasks": "To achieve the learning outcome, I recommend the following tasks: {taskList}"
        },
        "interviewQuestions": {
            "subjectQuery": "Welcome {user_name}, what do you want to learn today?",
            "timeQuery": "How much time do you have for this session?",
            "expertiseLevelQuery": "Please specify your current level of understanding: beginner, novice, or expert.",
            "depthQuery": "How deep would you like to delve into this topic? (surface-level overview, detailed study, or comprehensive deep dive)"
        },
        "taskStructure": {
            "instruct": "I will present the material related to '{sessionTitle}' in a structured manner, covering key concepts and providing real-world examples.",
            "test": "After the instruction, I'll assess your understanding with a brief test. Aim for at least 95% accuracy to ensure thorough comprehension.",
            "synthesize": "Finally, I'll prompt you to synthesize new information based on what you've learned. This will help in solidifying your understanding and encourage creative application of the knowledge."
        }
    },
    "sessions": [
        {
            "session_id": "1",
            "timestamp": "{timestamp_of_session}",
            "feedback": "{feedback_provided_by_user}"
        }
    ],
    "tools": {
        "INIT": {
            "description": "Initialize the AI agent and provide an overview of its capabilities.",
            "actions": [
                "Hello! I'm your iQ Human-AI collaboration partner. My primary objective is to assist you in your learning endeavors, guiding you through tailored instructional sessions, testing your understanding, and helping you synthesize new information. Together, we'll aim to achieve a comprehensive and retained understanding of your desired subjects at a minimum of 95% accuracy. I'm equipped with a range of tools and functions designed to offer a seamless, interactive experience. Please guide me with the commands provided in the configuration, and I'll do my best to assist you. Let's embark on this enlightening journey together!"
            ]
        },
        "USER_CONFIG": {
            "description": "Configure user information and preferences for the session.",
            "actions": [
                "{execute: interviewQuestions.subjectQuery}",
                "{execute: interviewQuestions.timeQuery}",
                "{execute: interviewQuestions.expertiseLevelQuery}",
                "{execute: interviewQuestions.depthQuery}"
            ]
        },
        "BEGIN_LESSON": {
            "description": "Starts the lesson execution process.",
            "actions": [
                "{execute: collaboration.functions.determineTitle}",
                "{execute: collaboration.functions.formulateLearningOutcome}",
                "{execute: taskStructure.instruct}",
                "{execute: taskStructure.test}",
                "{execute: taskStructure.synthesize}"
            ]
        },
        "CONCLUDE_LESSON": {
            "description": "Concludes the lesson and gathers feedback.",
            "actions": [
                "Thank you for participating in this session. Let's wrap things up.",
                "Recording current timestamp for the session.",
                "Please provide your feedback on the session.",
                "{execute: collaboration.feedback}"
            ]
        }
    }
}


## Execution

In [ ]:
# INIT

In [ ]:
# User Config

In [ ]:
# Begin Lesson

In [ ]:
# Conclude Lesson

# Phase 2 Plan:

Transfer to Build Infrastructure

- FastAPI Uvicorn server, no front-end
  - classes:
    - UserProfile:
    - ManagementCOT: COT prompt curation for management of agents
      - init: init prompt for manager or specialized agent
      - query: COT prompt curation for queries to manager or specialized agent, needs
  - Endpoints:
    - api/start @PUT --> receives str: "start", begins get_user_profile method
    - api/start @POST --> receives str: user_response
    - api/start @GET --> loads user  profile checkpoints
  - methods:
    - audio_output(response_text)
    - audio_input():
      - open connection for steaming input from mic at user command
      - store in user_input
      - close connection
      - return user_input
    - get_user_profile:
      - instantiate UserProfile class
      - user_profile.interview = dict: (str: question, str: response)
      - for question in userprofile.interview
        - current_question = question
        - current_question_audio = text2speech(current_question)
        - audio_output(current_question_audio)
        - user_input = audio_input()
        - user_profile.interview.currentquestion = speech2text(user_input)
    - manage_agent(str: agent_selection, str: prompt_type)
      - switch case: use agent_selection variable as condition to determine which agent to initialize, i.e. manager or specialized --> set isManager to true or false
      - switch case: use prompt_type variable as condition to determine which type of prompting request to ssend, i.e. init or query --> set isInit to true or false    
      - if isInit:
          response = send_request(class_selection.init_prompt)
        else:
          request = encapsulate_query(class_selection.query_prompt)
          response = send_request(request)
    - encapsulate_query(capsule)
      - user_request = audio_input
      - prompt = capsule + user_request
      - send_request(prompt)
    - store_profile_checkpoints
    - load_profile_checkpoints
- IBM Cloud integration
  - Docker Containerization: Dockerfile, docker-compose.yml
  - Create cluster on Kubernetes Container Registry <-- needs Research
  - Deploy container on Ubuntu Virtual Server <-- needs Research



## Workflow

- Create comprehensive, step-by-step detailed action-oriented subject matter lessons in the forms of tasks with high granularity

## Get Context

In [ ]:
def capture_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True)
    frames = []
    while True:
        data = stream.read(1024)
        frames.append(data)
        if len(frames) >= 100:
            break
    stream.stop_stream()
    stream.close()
    p.terminate()
    return frames

def transcribe_audio(frames):
    client = deepgram.Client()
    transcript = client.transcribe(audio_data=b''.join(frames))
    return transcript.get_text()

def interact_with_watson(prompt_text):
    # Create the foundation model object
    foundation_model = foundation_models.Model(model_id="google/flan-t5-xxl")

    # Generate a response using Watson
    response = foundation_model.generate(prompt=prompt_text)
    return response["generated_text"]

def main_interaction():
    audio_frames = capture_audio()
    user_prompt = f"Encapsulated prompt text {transcribe_audio(audio_frames)}"
    watson_response = interact_with_watson(user_prompt)
    # @TODO: Convert watson_response to audio and play it
    return watson_response


### Text to Speech --> interview questions

### audio output

### audio input

### Speech to Text --> user responses

### refine

- create task from response
- prioritize
- store in user_profile.burndown_list

### iterate

## Generate Output

### Select task based on user_profile.user_availability and user_profile.burndown_list --> selected_task

### Encapsulate user_profile.burndownlist.selected_task within Init Prompt for AI agent --> user_prompt

### Tokenize and send request to Watson

### Text to Speech <-- response

### Audio output <-- response

### Store Checkpoints <-- response